In [46]:
using LinearAlgebra
using ITensors
using Plots

### Canonical Forms



In [ ]:
# parameters 
N = 10
d = 3
D = 20

# random MPS
#=
    Order of legs: left-bottom-right
    Note: this is the conventional order used for MPSs in the code.
=#

Mrand = []
push!(Mrand, rand(1, d, D))
for l in 2:N-1
    push!(Mrand, rand(D, d, D))
end
push!(Mrand, rand(D, d, 1))


10-element Vector{Any}:
 [0.5422362190514134 0.7845013243823326 0.6507574576582956;;; 0.09179036528532691 0.6136887800481536 0.5614754540058613;;; 0.8423146057998141 0.033112492034494956 0.6150101764630789;;; … ;;; 0.6328515129172616 0.15740725408940937 0.9939154009266253;;; 0.7225737636959885 0.0930811738096976 0.2412852720485409;;; 0.8259684297912958 0.35047027293618527 0.0537841284130014]
 [0.44681309078409126 0.593282296145087 0.23992036611266032; 0.05568457571597052 0.9147245294157866 0.8615381636890911; … ; 0.04204878881493501 0.5878240425968217 0.747805962662747; 0.39873998226591856 0.08753140420876082 0.9794086077363789;;; 0.04136278276348415 0.5072030734264927 0.6809757017294586; 0.30736914917206637 0.32430709942117864 0.6101938261961283; … ; 0.22767170914984625 0.22895917239314545 0.04650340651206253; 0.4580039613029113 0.1544087533924563 0.014927279848892305;;; 0.2980094151289675 0.6136746319564398 0.12080093161989647; 0.17974146251901701 0.917402010715491 0.5259824950879337

In [60]:
#=
    Function that takes an MPS 'M' as input and returns a copy of it that is
    transformed into left canonical form and normalized.
=#
function left_canonical(M::Vector{Array{Float64,3}})
    Mcopy = deepcopy(M)
    N = length(Mcopy)

    for l in 1:N-1
        A = Mcopy[l]
        (d1, d2, d3) = size(A)
        A_mat = reshape(A, d1 * d2, d3)
        U, S, Vt = svd(A_mat; full=false)
        chi = size(U, 2)
        Mcopy[l] = reshape(U, d1, d2, chi)

        # Propagate S*Vt into next tensor
        next_A = Mcopy[l+1]
        tmp = S * Vt
        tmpA = reshape(tmp, chi, 1, size(tmp, 2))  # add dummy middle leg for einsum
        # Contract tmp with next_A over last index of tmp and first of next_A
        Mcopy[l+1] = reshape(sum(tmpA .* permutedims(next_A, (2,3,1)), dims=3), size(tmpA,1), size(next_A,2), size(next_A,3))
    end

    return Mcopy
end


left_canonical (generic function with 3 methods)